In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime, timedelta
import csv

## Objective:
-Import the datasets. Still need the extra dataset. Includes triggers up to dec. 21

-datetime helpful docs: https://www.w3schools.com/python/python_datetime.asp

-Create algorithm that searches for matches that are +/- 10 second apart and remove the rest (maybe two seconds)

-First match found at 2018, 7, 11, 17, 2, 4

-Complete match list found for trigB --> trigB_match

//TODO: 

- Update list up until 2022
- Wrap functions around the algorithms so that the same can be done for trigC etc.
- Create dataframe with the result
-Access the relevant files found in trigB_match list
-Do spectral analysis on the data (counts/bin)


### Notes from stackoverflow:
np.where() implementation:https://numpy.org/doc/stable/reference/generated/numpy.where.html , https://stackoverflow.com/questions/35714902/find-location-of-pair-of-elements-in-two-arrays-in-numpy

## Importing data

In [67]:
#Path to data
IPN_path = "/Home/siv30/wad005/master/GRB/IPN"
ICECUBE_path = "/Home/siv30/wad005/master/GRB/IPN"
ASIM_path = "/Home/siv30/wad005/master/GRB/ASIM"

#Path to export data
IPN_match_path = "/Home/siv30/wad005/master/GRB/matches_IPN"
ICECUBE_match_path = "/Home/siv30/wad005/master/GRB/matches_ICECUBE"

In [19]:
#Importing and redefining the dataframes
#Not loading the trigger list for 2021 just yet

#Old IPN dataset
ipn_old_data = pd.read_csv("{}/trigIPN.csv".format(IPN_path), sep="|")
ipn_old_df = pd.DataFrame(ipn_old_data)
ipn_old_df.drop(columns= ['Unnamed: 0', 'Unnamed: 2'], axis=1, inplace=True) #dropping unwanted columns
ipn_old_df.rename(columns={ipn_old_df.columns[0]:"datetime"}, inplace = True)

#New IPN dataset
ipn_new_data = pd.read_csv("{}/icecube.txt".format(ICECUBE_path),sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
ipn_new_df = pd.DataFrame(ipn_new_data)
#Dropping unwanted columns
ipn_new_df = ipn_new_df.drop(["GRB_name_Fermi",'ra(°J2000)', 'decl(°J2000)',
       'pos_error(°1-sigma)', 'T90(s)', 'T90_error(s)', 'T90_start(UTC)',
       'fluence(erg/cm^2)', 'fluence_error(erg/cm^2)', 'redshift', 'T100(s)',
       'GBM_located', 'mjd(T0day)'], 1)

#HED 
trigB_data = pd.read_csv("{}/trigB.txt".format(ASIM_path), sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigB = pd.DataFrame(trigB_data)
trigB.drop(columns=["######"], inplace=True) #dropping unwanted columns
trigB.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr"}, inplace=True) #renaming columns

trigB_21_data = pd.read_csv("{}/trigB_2021.txt".format(ASIM_path), sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigB_21 = pd.DataFrame(trigB_21_data)
trigB_21.drop(columns=["######"], inplace=True)
trigB_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" ,"Corr":"corr"}, inplace=True)

#LED
trigC_data = pd.read_csv("{}/trigC.txt".format(ASIM_path), sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigC = pd.DataFrame(trigC_data)
trigC.drop(columns=["######"], inplace=True)
trigC.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" ,"Corr":"corr"}, inplace=True)

trigC_21_data = pd.read_csv("{}/trigC_2021.txt".format(ASIM_path), sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigC_21 = pd.DataFrame(trigC_21_data)
trigC_21.drop(columns=["######"], inplace=True)
trigC_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", 'Corr':"corr" }, inplace=True)

#MMIA (visible)
trigM_data = pd.read_csv("{}/trigM.txt".format(ASIM_path), sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigM = pd.DataFrame(trigM_data)
trigM.drop(columns=["######"], inplace=True)
trigM.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr" }, inplace=True)

trigM_21_data = pd.read_csv("{}/trigM_2021.txt".format(ASIM_path), sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigM_21 = pd.DataFrame(trigM_21_data)
trigM_21.drop(columns=["######"], inplace=True)
trigM_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr" }, inplace=True)

## Concatenating ASIM dataframes

In [27]:
def find_index_21(l):
    """
    Function for finding the indecies to slice. Assuming l is sorted by date from first to last.
    Finding index of first entry of '2021-Jan-01'.
    
    Parameters:
    -----
    l <array>
    
    Returns:
    -----
    Returning index for the first entry of '2021-Jan-01' """
    
    
    i = l.tolist().index('2021-Jan-01')
    return i

In [28]:
slice_index = []
for df in [trigB,trigC,trigM]:
    temp_date_list = df["date"].values
    slice_index.append(find_index_21(temp_date_list))
slice_index

[110410, 79065, 237671]

In [29]:
#trig*_20 contains rows to the end of 2020
#trig*_master contains rows to the end of 2021
trigB_20 = trigB.drop(np.arange(slice_index[0],len(trigB))) 
trigB_long = pd.concat([trigB_20,trigB_21])

trigC_20 = trigC.drop(np.arange(slice_index[1],len(trigC)))
trigC_long = pd.concat([trigC_20,trigC_21])

trigM_20 = trigM.drop(np.arange(slice_index[2],len(trigM)))
trigM_long = pd.concat([trigM_20,trigM_21])

## IPN Handling

### IPN_old

In [31]:
#Old IPN datetime list
ipn_dt_temp = []
ipn_dict = ipn_old_df.to_dict("records")
for row in ipn_dict:
    datetime_str = row["datetime"]
    datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    ipn_dt_temp.append(datetime_obj)
    
ipn_old_dt = np.asarray(ipn_dt_temp)
ipn_old_dt.shape

(1176,)

### IPN new

In [32]:
# Zero-padding hour in times

def zero_padding(l):
    """Function for zero padding times in IPN new
    Parameters:
    -----
    l <array> containing string of times

    Returns:
    -----
    new_times <array> containing string of zero-padded times
    """
    new_times = []

    for time in l:
        if time.index(":") < 2:
            time = "0" + time
            new_times.append(time)
        else:
            new_times.append(time)
    return np.array(new_times)

In [33]:
#Vectorization of names and times from dataframe.
names = ipn_new_df["GRB_name"].values
times = ipn_new_df["T0(UTC)"].values  # Hour is not zero padded.
new_times = zero_padding(times)

In [34]:
# Defining datetime objects in ipn_new_list.
# Problem: some of the times have microsecond resolution while other dont
ipn_new_list = []
if len(names) == len(new_times):
    for i in np.arange(0, len(names)):
        date = names[i][3:9]
        time = new_times[i]

        # Managing the microsecond resolution
        if len(time) > 8:
            dt_obj = datetime.strptime(date + " " + time, "%y%m%d %H:%M:%S.%f")
        else:
            dt_obj = datetime.strptime(date + " " + time, "%y%m%d %H:%M:%S")
            
        ipn_new_list.append(dt_obj)
else:
    print("Unequal length of arrays")
    
#Earliest entry in the ASIM data 2018,6,1,12,46,8
# Including only entries up to 1142. That means entries in IPN starts from 2018, 5, 29, 8, 29, 14

ipn_new_dt = np.asarray(ipn_new_list[:1142][::-1]) #Slicing the list. Returning in reverse order (decending)


In [ ]:
#Function for stripping the microseconds from an datetime object. Not necessary
def strip_microseconds(l):
    """
    Strip microseconds from datetime objects in list containing a tuple of dt objects and int. Returning string
    objects. Seperating them in two arrays
    
    Parameters
    ----------
    l : list of datetime objects
        The list containing the objects.
    
    Returns
    -------
    updated_list : array of stripped datetime objects as string and array of trig_id
    
    Raises
    ------
    TypeError
        If the input is not a type list or array
    """
    time = []
    trig_id = []
    try:
        for tup in l:
            dt = tup[0].strftime("%Y-%m-%d %H:%M:%S")
            time.append(dt)
            trig_id.append(tup[1])
            
        return np.array((time,trig_id))
    
    except TypeError:
        print("Input is not the correct type <List> or <nd array> or entries is not tuple")

In [ ]:
#Important to keep the microseconds? Check the search algo
ipn_old_updated = strip_microseconds(matches_old)
ipn_new_updated = strip_microseconds(matches_new)

## ASIM datetime correction, vectorization

In [35]:
# Function for correcting date and time in ASIM data. Returning ndarray that contains datetime objects

def corr_dt(dfs):  # Correcting times from ASIM data and returning nested array of datetime objects
    
    """   This function corrects the time from ASIM data and returns datetimeobjects
    The correction is done by subtracting the correction from the original time.
    The correction is given as a string.
    The function takes a list of dataframes as input.
    The function returns a list of arrays containing the corrected datetime objects.
    
    Parameters
    ----------
    MAYBE LISTS CONTAINING STRING OF tuple(DATE,TIME)?
    
    dfs : list of dataframes
        The dataframes containing the data.
    
    Returns
    -------
    trig_dt : list of arrays
        The corrected datetime objects.
        trig_dt[0] --> trigB
        trig_dt[1] --> trigC
        trig_dt[2] --> trigM
    
    Raises
    ------
    ValueError
        If the lists are not the same length.
    """
    trig_dt = []
    
    for df in dfs:
        temp_dt = []
        # Vectorization of columns
        date = df["date"].values  # date given as string.
        time = df["time"].values  # time given as string
        corr = df["corr"].values  # correction given as string
        try:
            if len(date) and len(time) != len(corr):
                raise ValueError
        except:
            raise ValueError("Lists are not the same length")
        else:
            for i in np.arange(0, len(corr)):  # Iterating over the vectors
                if corr[i] == "--------":  # No correction needed. Appending the datetime object
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")
                    temp_dt.append(org_dt)
                    
                elif corr[i][0] == "-":  # If it's a "-" in front; correction is added
                    # formatting the datetime object
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")  # Original datetime

                    micro_corr = int(corr[0][1:])

                    # new corrected datetime. Timedelta ccounts for changes in seconds also
                    new_dt = org_dt + timedelta(microseconds=micro_corr)
                    temp_dt.append(new_dt)
                else:
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")  # Original datetime

                    micro_corr = int(corr[0][1:])

                    # new corrected datetime. Timedelta ccounts for changes in seconds also
                    new_dt = org_dt - timedelta(microseconds=micro_corr)
                    temp_dt.append(new_dt)
                    
            trig_dt.append(temp_dt)

    trig_dt = np.array(trig_dt, dtype=object)
    return trig_dt

In [36]:
#Callig corr_dt with a list containing the dataframes from ASIM
trig_master = corr_dt([trigB_long,trigC_long,trigM_long])

#Creating array instances in trig_dt
for i in np.arange(0,len(trig_master)):
    trig_master[i] = np.asarray(trig_master[i])

## ASIM and IPN search

In [40]:
#Most compact algorithm for searching: np.where()
#Storing matches in the match list. Stored as a tuple containing (datetime IPN, index trig_B)

# TODO: wrap a function around it so it can take in several triggers (trigB,trigC etc..)
def find_match(trig_arr):
    """
    Parameters:
    ------
    trig_arr <array> <datetime>
    np.where returns: An array with elements from x where condition is True, and elements from y elsewhere.
    
    elements in matches_* contains tuples for indecies in IPN list and trig_arr. """
    matches_old = [] 

    for i in ipn_old_dt:
        #Searching for matches that are +/- 10 seconds from the IPN trigger
        mask = np.where((i-timedelta(seconds=10) <= trig_arr) & (trig_arr <= i + timedelta(seconds=10)))
        if len(mask[0]) == 0:
            continue
        else:
            matches_old.append((i,mask[0]))

    matches_new = []
    for i in ipn_new_dt:
        #Searching for matches that are +/- 10 seconds from the IPN trigger
        mask = np.where((i-timedelta(seconds=10) <= trig_arr) & (trig_arr <= i + timedelta(seconds=10)))
        #print(mask)
        if len(mask[0]) == 0:
            continue
        else:
            matches_new.append((i,mask[0]))
        
    return (matches_old, matches_new)

In [41]:
master_match = []
for i in trig_master:
    tup_match = find_match(i)
    master_match.append(tup_match)

In [47]:
len(master_match[0][0])

34

## Branching
Branching into matches in old ipn dataset and matches in new ipn dataset.

Export into a dataframe containing IPN dt trigger | trig* dt trigger



How to access data from master_match:


- master_match[n] - refers to type of trigger (trigB, trigC, trigM)

- master_match[n][n] - refers to old or new ipn trigger list [0]: old , [1]: new

- master_match[n][n][n] - refers to tuple containing (ipn dt, array[index]) where index is the dt in trig_dt[n][index]

- master_match[n][n][n][n] - selecting desired element in tuple

### Branching into seperate arrays

In [48]:
match_trigB_old = np.array(master_match[0][0],dtype=object)
match_trigB_new = np.array(master_match[0][1],dtype=object)

match_trigC_old = np.array(master_match[1][0],dtype=object)
match_trigC_new = np.array(master_match[1][1],dtype=object)

match_trigM_old = np.array(master_match[2][0],dtype=object)
match_trigM_new = np.array(master_match[2][1],dtype=object)

### Finding datetime object from index

In [103]:
def index_to_dt(i,arr_index):
    """Retrives the datetime object specified.
    Parameters:
    -------
    i <int> index in the trig_dt array. Specifying which trigger to access
    arr_index <array><int> index of the datetime object in the array
    
    Returns:
    ------
    <datetime> the specified datetimeobject"""
    dts = []
    for index in arr_index:
        dt = trig_master[i][index]
        dts.append(dt)
    return dts

## Creating new dataframes with matches

In [104]:
# Creating dataframe for ipn old and ipn new. Columns should be ipn dt trigger, trigB, trigC,trigM
# if there isnt a trigger in the respected trigger, the entry should be None
# But first. Produce df of ipn --> trigB, trigC, trigM [indecies]. Just indecies because there are several in some triggers

trigB_old_dict = {"ipn old dt":[], "trigB index": []}
for i in match_trigB_old:
    trigB_old_dict["ipn old dt"].append(i[0])
    trigB_old_dict["trigB index"].append(i[1])
trigB_old_df = pd.DataFrame(trigB_old_dict)

trigB_new_dict = {"ipn new dt":[], "trigB index": []}
for i in match_trigB_new:
    trigB_new_dict["ipn new dt"].append(i[0])
    trigB_new_dict["trigB index"].append(i[1])
trigB_new_df = pd.DataFrame(trigB_new_dict)

trigC_old_dict = {"ipn old dt":[], "trigC index": []}
for i in match_trigC_old:
    trigC_old_dict["ipn old dt"].append(i[0])
    trigC_old_dict["trigC index"].append(i[1])
trigC_old_df = pd.DataFrame(trigC_old_dict)

trigC_new_dict = {"ipn new dt":[], "trigC index": []}
for i in match_trigC_new:
    trigC_new_dict["ipn new dt"].append(i[0])
    trigC_new_dict["trigC index"].append(i[1])
trigC_new_df = pd.DataFrame(trigC_old_dict)

trigM_old_dict = {"ipn old dt":[], "trigM index": []}
for i in match_trigM_old:
    trigM_old_dict["ipn old dt"].append(i[0])
    trigM_old_dict["trigM index"].append(i[1])
trigM_old_df = pd.DataFrame(trigM_old_dict)

trigM_new_dict = {"ipn new dt":[], "trigM index": []}
for i in match_trigM_new:
    trigM_new_dict["ipn new dt"].append(i[0])
    trigM_new_dict["trigM index"].append(i[1])
trigM_new_df = pd.DataFrame(trigM_new_dict)

### Appending datetime objects as string

#### IPN df

In [105]:
trigB_old_indecies = trigB_old_df["trigB index"].values
trigB_old_dts = index_to_dt(0,trigB_old_indecies)

trigC_old_indecies = trigC_old_df["trigC index"].values
trigC_old_dts = index_to_dt(1,trigC_old_indecies)

trigM_old_indecies = trigM_old_df["trigM index"].values
trigM_old_dts = index_to_dt(2,trigM_old_indecies)


In [106]:
trigB_old_df["trigB dt"] = trigB_old_dts
trigC_old_df["trigC dt"] = trigC_old_dts
trigM_old_df["trigM dt"] = trigM_old_dts

####  ICECUBE df


In [107]:
trigB_new_indecies = trigB_new_df["trigB index"].values
trigB_new_dts = index_to_dt(0,trigB_new_indecies)

trigC_new_indecies = trigC_new_df["trigC index"].values
trigC_new_dts = index_to_dt(1,trigC_new_indecies)

trigM_new_indecies = trigM_new_df["trigM index"].values
trigM_new_dts = index_to_dt(2,trigM_new_indecies)

In [108]:
trigB_new_df["trigB dt"] = trigB_new_dts
trigC_new_df["trigC dt"] = trigC_new_dts
trigM_new_df["trigM dt"] = trigM_new_dts

In [110]:
trigC_new_df

,ipn old dt,trigC index,trigC dt
0,2018-12-27 06:16:59,[11202],[2018-12-27 06:17:04.210500]
1,2019-01-17 08:50:43,[12923],[2019-01-17 08:50:42.939689]
2,2019-04-20 23:32:24,[20754],[2019-04-20 23:32:24.931211]
3,2019-05-01 05:23:21,[21437],[2019-05-01 05:23:21.736797]
4,2019-06-15 14:42:25,[23861],[2019-06-15 14:42:22.741579]
5,2019-06-28 04:23:39,[24588],[2019-06-28 04:23:32.512290]
6,2019-08-13 12:00:45,[27017],[2019-08-13 12:00:48.801260]
7,2019-08-25 21:04:56,"[27707, 27708]","[2019-08-25 21:04:53.870393, 2019-08-25 21:04:..."
8,2019-09-06 01:04:53,[28416],[2019-09-06 01:04:52.014004]
9,2019-10-01 06:41:50,[30103],[2019-10-01 06:41:51.631214]


## Exporting matches from IPN

### trigB export

In [ ]:
for index, row in trigB_old_df.iterrows():
    trigB_old_df["trigB dt"].iloc[index] = []
    for i in row[2]:
        trigB_old_df["trigB dt"].iloc[index].append(datetime.strftime(i,"%Y-%m-%d %H:%M:%S.%f"))

In [ ]:
trigB_old_df.to_csv("{}/trigB_old_match.csv".format(IPN_match_path))

### trigC export

In [70]:
for index, row in trigC_old_df.iterrows():
    trigC_old_df["trigC dt"].iloc[index] = []
    for i in row[2]:
        trigC_old_df["trigC dt"].iloc[index].append(datetime.strftime(i,"%Y-%m-%d %H:%M:%S.%f"))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [72]:
trigC_old_df.to_csv("{}/trigC_old_match.csv".format(IPN_match_path))

### trigM export

In [ ]:
for index, row in trigM_old_df.iterrows():
    trigM_old_df["trigM dt"].iloc[index] = []
    for i in row[2]:
        trigM_old_df["trigM dt"].iloc[index].append(datetime.strftime(i,"%Y-%m-%d %H:%M:%S.%f"))

In [ ]:
trigM_old_df.to_csv("trigM_old_match.csv")

### trigger lists

In [ ]:
trigB_strf = []
for dt in trig_dt[0]:
    strf = dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    trigB_strf.append(strf)
    
    
trigB_df_exp = pd.DataFrame({"datetime": trigB_strf})
trigB_df_exp.to_csv("HED triggers.csv")

In [ ]:
trigC_strf = []
for dt in trig_dt[1]:
    strf = dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    trigC_strf.append(strf)
    
    
trigC_df_exp = pd.DataFrame({"datetime": trigC_strf})
trigC_df_exp.to_csv("LED triggers.csv")

In [ ]:
trigM_strf = []
for dt in trig_dt[2]:
    strf = dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    trigM_strf.append(strf)
    
    
trigM_df_exp = pd.DataFrame({"datetime": trigM_strf})
trigM_df_exp.to_csv("MMIA triggers.csv")

## ICECUBE export

In [112]:
#Formatting datetimeobjects to strings
for index, row in trigB_new_df.iterrows():
    trigB_new_df["trigB dt"].iloc[index] = []
    for i in row[2]:
        trigB_new_df["trigB dt"].iloc[index].append(datetime.strftime(i,"%Y-%m-%d %H:%M:%S.%f"))
        
trigB_new_df.to_csv("{}/HED_new_match.csv".format(ICECUBE_match_path))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [113]:
#Formatting datetimeobjects to strings
for index, row in trigC_new_df.iterrows():
    trigC_new_df["trigC dt"].iloc[index] = []
    for i in row[2]:
        trigC_new_df["trigC dt"].iloc[index].append(datetime.strftime(i,"%Y-%m-%d %H:%M:%S.%f"))
        
trigC_new_df.to_csv("{}/LED_new_match.csv".format(ICECUBE_match_path))



In [115]:
#Formatting datetimeobjects to strings
for index, row in trigM_new_df.iterrows():
    trigM_new_df["trigM dt"].iloc[index] = []
    for i in row[2]:
        trigM_new_df["trigM dt"].iloc[index].append(datetime.strftime(i,"%Y-%m-%d %H:%M:%S.%f"))
        
trigM_new_df.to_csv("{}/MMIA_new_match.csv".format(ICECUBE_match_path))

### Graveyard

In [ ]:
"""Function to filter duplicates
def filter_duplicates(l1,l2):
    
    l1 --> ipn_old match
    l2 --> ipn_new match
    
    Converting l1,l2 to np.arrays to handle the filtering. (might not be neccecery)
    Checking if dt object in l2 is not present in l1. If so, then add the tuple to l1
    Parameters: l1,l2 <list> containing tuples
    ----
    Returns: l3 <array> of dt objects that are not in 
    
    found_time = []
    indecies = []
    for index, i in enumerate(ipn_new_updated[0]):
        if i not in ipn_old_updated:
            found_time.append(i)
            indecies.append(index)
        
    found_time = np.asarray(found_time)
    indecies = np.asarray(indecies) """